In [1]:
import git  #Requires Gitpython installed, to install do "$ pip install Gitpython" in your terminal
import pkg_resources
openfisca_france_location = pkg_resources.get_distribution('openfisca-france').location
repo = git.Repo(openfisca_france_location)
repo.git.status()

sha = repo.head.object.hexsha

print "---This notebook have been runed with--- \n","Openfisca-france :", repo.head.object.hexsha, '\n' 
print "Commit short SHA:", repo.git.rev_parse(sha, short=4)
print "Branch: ", repo.git.rev_parse('--abbrev-ref', "HEAD") #equivalent to: $git rev-parse --abbrev-ref HEAD
print "Date (y/m/d):", repo.git.show("-s", '--format=%ci', 'HEAD^')[:-5],"\n"  # equivalent to: $git show -s --format=%ci HEAD^

print "Openfisca-core :", git.Repo(pkg_resources.get_distribution('openfisca-core').location).head.object.hexsha, ' Date:', git.Repo(pkg_resources.get_distribution('openfisca-core').location).git.show("-s", '--format=%ci', 'HEAD^')[:-5]
print "Openfisca-france-data :", git.Repo(pkg_resources.get_distribution('openfisca-france-data').location).head.object.hexsha, ' Date:', git.Repo(pkg_resources.get_distribution('openfisca-france-data').location).git.show("-s", '--format=%ci', 'HEAD^')[:-5]
print "OpenFisca-Aggregates :", git.Repo(pkg_resources.get_distribution('OpenFisca-Aggregates').location).head.object.hexsha, ' Date:', git.Repo(pkg_resources.get_distribution('OpenFisca-Aggregates').location).git.show("-s", '--format=%ci', 'HEAD^')[:-5]

---This notebook have been runed with--- 
Openfisca-france : f667d205fac835e41ef3feaaf251ab0cd1df75b7 

Commit short SHA: f667d
Branch:  heads/remotes/openfisca/master
Date (y/m/d): 2016-02-24 16:12:06  

Openfisca-core : 4076f591f4bcb571391b65a989e1603fdc4cb759  Date: 2016-02-24 16:11:57 
Openfisca-france-data : 60e1891a030ce9c854fe24a961e06e4e0207feaa  Date: 2016-02-27 16:45:24 
OpenFisca-Aggregates : 79715d009b35180d9cfd542fee778042ea950d55  Date: 2016-02-15 17:42:07 


In [54]:
from __future__ import division
from openfisca_core import periods
from openfisca_plugin_aggregates.tests.reform.test_aggregates_mensualized import create_survey_scenario
import pandas as pd

import numpy as np
from numpy import (datetime64, logical_and as and_, logical_not as not_, logical_or as or_, logical_xor as xor_,
maximum as max_, minimum as min_, round)


#Faire des filtres pour Mahdi survey_scenario.create_data_frame_by_entity_key_plural

pd.set_option('expand_frame_repr', False)  #utile quand on utilise pandas en mode console
pd.set_option("display.max_rows",201)
pd.set_option("display.max_columns",201)

In [55]:
period = periods.period(2009)

In [56]:
survey_scenario = create_survey_scenario(2009)

In [58]:
print survey_scenario.compute_aggregate('irpp')
print survey_scenario.compute_aggregate('irpp_mensuel')
print "l'impot mensuel mène à un prélèvement de {} euros en plus".format(survey_scenario.compute_aggregate('irpp_mensuel')-survey_scenario.compute_aggregate('irpp') )

-34324438907.3
-45448197454.2
l'impot mensuel mène à un prélèvement de -11123758546.9 euros en plus


In [59]:
print survey_scenario.weight_column_name_by_entity_key_plural
survey_scenario.simulation.calculate_add(survey_scenario.weight_column_name_by_entity_key_plural['foyers_fiscaux'], period = period).astype(float)

{'foyers_fiscaux': 'weight_foyers', 'individus': 'weight_individus', 'familles': 'weight_familles', 'menages': 'wprm'}


array([ 378.,  378.,  789., ...,  240.,  767.,  767.])

In [60]:
weight_foyers = survey_scenario.simulation.calculate_add('weight_foyers', period = period).astype(float)

### Stat de Base sur la différence

In [61]:
# We put positive value for income tax (negative in openfisca output)
impot_annuel =  -survey_scenario.simulation.calculate('irpp')
impot_mensuel =  -survey_scenario.simulation.calculate_add('irpp_mensuel')

In [62]:
impot_annuel = survey_scenario.simulation.calculate('irpp')
impot_mensuel = survey_scenario.simulation.calculate_add('irpp_mensuel')
print "il y a {}/{} cas ou l'impot annuel et mensuel est null soit {}%".format(((impot_annuel == 0) & (impot_mensuel == 0)).sum(),impot_mensuel.shape[0],((impot_annuel == 0) & (impot_mensuel == 0)).sum()/impot_mensuel.shape[0] )
print "il y a {}/{} cas ou l'impot mensuel est null, mais pas l'annuel soit {}% (pas possible ?)".format(((impot_annuel != 0) & (impot_mensuel == 0)).sum(), impot_mensuel.shape[0],((impot_annuel != 0) & (impot_mensuel == 0)).sum()/impot_mensuel.shape[0])
print "il y a {}/{} cas ou l'impot mensuel est inférieur à l'annuel soit {}% (pas possible ?)".format((impot_annuel > impot_mensuel).sum(), impot_mensuel.shape[0], (impot_annuel > impot_mensuel).sum()/ impot_mensuel.shape[0])


il y a 30864/59874 cas ou l'impot annuel et mensuel est null soit 0.515482513278%
il y a 0/59874 cas ou l'impot mensuel est null, mais pas l'annuel soit 0.0% (pas possible ?)
il y a 18890/59874 cas ou l'impot mensuel est inférieur à l'annuel soit 0.31549587467% (pas possible ?)


#### Annuel == mensuel 

In [63]:
print "il y a {}/{} cas ou l'impot mensuel est égal à l'annuel soit {}% ".format((np.abs(impot_annuel - impot_mensuel)==0).sum(), impot_mensuel.shape[0], (np.abs(impot_annuel - impot_mensuel)==0).sum()/ impot_mensuel.shape[0])
print "il y a {}/{} cas ou l'impot mensuel est égal à l'annuel (à 1 cents près) soit {}% ".format((np.abs(impot_annuel - impot_mensuel)<0.01).sum(), impot_mensuel.shape[0], (np.abs(impot_annuel - impot_mensuel)<0.01).sum()/ impot_mensuel.shape[0])
print "il y a {}/{} cas ou l'impot mensuel est égal à l'annuel (à 1 euros près) soit {}% ".format((np.abs(impot_annuel - impot_mensuel)<1).sum(), impot_mensuel.shape[0], (np.abs(impot_annuel - impot_mensuel)<1).sum()/ impot_mensuel.shape[0])
print "il y a {}/{} cas ou l'impot mensuel est égal à l'annuel (à 10 euros près) soit {}% ".format((np.abs(impot_annuel - impot_mensuel)<10).sum(), impot_mensuel.shape[0], (np.abs(impot_annuel - impot_mensuel)<10).sum()/ impot_mensuel.shape[0])

il y a 35067/59874 cas ou l'impot mensuel est égal à l'annuel soit 0.585679927848% 
il y a 46355/59874 cas ou l'impot mensuel est égal à l'annuel (à 1 cents près) soit 0.774209172596% 
il y a 46861/59874 cas ou l'impot mensuel est égal à l'annuel (à 1 euros près) soit 0.782660253198% 
il y a 47053/59874 cas ou l'impot mensuel est égal à l'annuel (à 10 euros près) soit 0.78586698734% 


####  mensuel < annuel (supposé être l'exception)

In [64]:
print "il y a {}/{} cas ou l'impot mensuel est inférieur à l'annuel soit {}% (pas possible ?)".format((impot_annuel > impot_mensuel).sum(), impot_mensuel.shape[0], (impot_annuel > impot_mensuel).sum()/ impot_mensuel.shape[0])
print "il y a {}/{} cas ou l'impot mensuel est inférieur à l'annuel (à 10 euros près) soit {}% (pas possible ?)".format((impot_annuel-impot_mensuel >10 ).sum(), impot_mensuel.shape[0], ((impot_annuel-impot_mensuel >10)).sum()/ impot_mensuel.shape[0])

il y a 18890/59874 cas ou l'impot mensuel est inférieur à l'annuel soit 0.31549587467% (pas possible ?)
il y a 12801/59874 cas ou l'impot mensuel est inférieur à l'annuel (à 10 euros près) soit 0.213798977853% (pas possible ?)


In [65]:
print "il y a {}/{} cas ou l'impot mensuel est inférieur à l'annuel (à 10 euros près) soit {}% (pas possible ?)".format((impot_annuel-impot_mensuel >10 ).sum(), impot_mensuel.shape[0], ((impot_annuel-impot_mensuel >10)).sum()/ impot_mensuel.shape[0])

il y a 12801/59874 cas ou l'impot mensuel est inférieur à l'annuel (à 10 euros près) soit 0.213798977853% (pas possible ?)


####  mensuel < annuel (supposé être la norme)

In [66]:
print "il y a {}/{} cas ou l'impot mensuel est supérieur à l'annuel soit {}% (pas possible ?)".format((impot_annuel < impot_mensuel).sum(), impot_mensuel.shape[0], (impot_annuel < impot_mensuel).sum()/ impot_mensuel.shape[0])
print "il y a {}/{} cas ou l'impot mensuel est supérieur à l'annuel (à 10 euros près) soit {}% (pas possible ?)".format((impot_mensuel-impot_annuel >10 ).sum(), impot_mensuel.shape[0], ((impot_mensuel-impot_annuel >10)).sum()/ impot_mensuel.shape[0])

il y a 5917/59874 cas ou l'impot mensuel est supérieur à l'annuel soit 0.0988241974814% (pas possible ?)
il y a 20/59874 cas ou l'impot mensuel est supérieur à l'annuel (à 10 euros près) soit 0.000334034806427% (pas possible ?)


In [ ]:
irpp_mois_list = 'ir'

## Différence annuel vs. mensuel

### Perdants

In [ ]:
simulation.calculate

####  mensuel < annuel

In [ ]:
survey_scenario.weight_column_name_by_entity_key_plural

### Nul du à la décote

In [41]:
decote = survey_scenario.simulation.calculate_add('decote')
ir_plaf_qf = survey_scenario.simulation.calculate_add('ir_plaf_qf')

decote_m = survey_scenario.simulation.calculate_add('decote_mensuel_times_12')
ir_plaf_qf_m = survey_scenario.simulation.calculate_add('ir_plaf_qf_mensuel_times_12')

decote = min_(decote, ir_plaf_qf)
decote_m = min_(decote_m, ir_plaf_qf)

In [42]:
irpp_mois_list = ['irpp_mois{}'.format(month) for month in range(1,13)]

### DIfférence de décote

In [43]:
(survey_scenario.compute_aggregate('decote_mensuel_times_12')/12) - (survey_scenario.compute_aggregate('decote'))

102872205.70319176

## ANnalyse

In [44]:
df = survey_scenario.create_data_frame_by_entity_key_plural(['decote_gain_fiscal','decote_gain_fiscal_mensuel_times_12',"irpp",'irpp_mensuel'])['foyers_fiscaux']

In [45]:
irpp_month = survey_scenario.simulation.calculate('irpp_mensuel', '2009-{}'.format(month))

for month in range(1,13):
    irpp_month = survey_scenario.simulation.calculate('irpp_mensuel', '2009-{}'.format(month))
    irpp_serie = pd.Series(irpp_month, name = 'irpp_{}'.format(month))
    df = df.T.append(irpp_serie).T

In [46]:
df.T

0      1      2      3            4           5            6            7      8      9           10           11          12     13     14           15          16          17           18           19           20     21          22          23     24           25           26           27     28     29     30           31           32          33     34          35     36           37          38           39           40           41           42           43           44           45           46     47     48     49     50           51           52     53           54     55           56           57     58     59          60     61     62     63     64           65            66           67          68           69     70          71          72     73           74            75     76     77     78     79           80          81          82          83     84     85     86     87          88     89     90           91     92     93     94     95           96           97            98     99     ...    59774  59775        59776        59777       59778        59779       59780        59781  59782  59783  59784        59785       59786        59787        59788  59789  59790  59791  59792  59793  59794        59795        59796        59797  59798  59799        59800  59801        59802  59803       59804  59805       59806        59807        59808        59809        59810  59811        59812        59813       59814  59815  59816  59817  59818  59819  59820       59821       59822        59823  59824        59825  59826        59827  59828     59829        59830        59831  59832  59833       59834       59835  59836  59837  59838  59839        59840       59841        59842  59843  59844  59845        59846        59847  59848  59849  59850        59851        59852  59853       59854  59855  59856  59857        59858         59859        59860  59861  59862  59863        59864        59865  59866        59867  59868         59869  59870       59871  59872  59873
decote_gain_fiscal                       0      0      0      0    26.372473   24.462494    -0.000000   244.184998      0      0    0.000000    -0.000000    0.000000      0      0    -0.000000  116.709999   -0.000000    -0.000000    -0.000000    -0.000000      0   -0.000000   -0.000000      0    -0.000000    -0.000000   144.092560      0      0      0    -0.000000    -0.000000    0.000000      0   -0.000000      0    -0.000000    0.000000    -0.000000   236.292496    -0.000000    -0.000000   121.193787    -0.000000    -0.000000    -0.000000      0      0      0      0   115.555054   121.989998      0   279.206268      0   215.378738   121.699951      0      0   -0.000000      0      0      0      0   229.982498     -0.000000    -0.000000    0.000000    -0.000000      0   -0.000000   -0.000000      0   133.872498     -0.000000      0      0      0      0    -0.000000   -0.000000   -0.000000   -0.000000      0      0      0      0   51.114990      0      0    -0.000000      0      0      0      0   172.452438    -0.000000     -0.000000      0  ...        0      0   224.455048   148.830002     0.00000   233.638763   -0.000000    -0.000000      0      0      0    -0.000000   -0.000000   199.462494    -0.000000      0      0      0      0      0      0   181.484970     0.000000    -0.000000      0      0   161.425034      0    -0.000000      0  146.332489      0    0.000000   262.817535    57.625000    -0.000000   120.230110      0    -0.000000    -0.000000  278.862488      0      0      0      0      0      0   -0.000000    0.000000    -0.000000      0    57.082489      0   133.278625      0  0.000000    -0.000000   247.773758      0      0    6.446228    0.000000      0      0      0      0     0.000000   15.292633    -0.000000      0      0      0    -0.000000   163.307495      0      0      0    -0.000000   140.195007      0    0.000000      0      0      0    -0.000000     -0.000000   268.234985      0      0      0    -0.000000   214.390060      0     

### Droppe les cas où la différence d'impôt est inférieure à 10 euros

In [47]:
import numpy as np

In [48]:
df_ir_1 = df[(np.abs(df.irpp - df.irpp_mensuel) - 5)> 0 ]

In [49]:
df_ir_1.shape

(12927, 16)

In [50]:
(df.decote_gain_fiscal> df.decote_gain_fiscal_mensuel_times_12/12).shape

(59874,)

### Droppe les cas où l'impôt mensuel est supérieur à l'annuel

In [51]:
df_ir_sup_ir_m = df_ir_1[df_ir_1.irpp < df_ir_1.irpp_mensuel ]

In [52]:
df_ir_sup_ir_m.T

1319        2701        3186         3260         5609        9026         12360        14581       15745        19754       22808        23599        23938        33728        33849       36120        38816        39517        42292       45166        47299        47992        48149       50743        51189        52656        53797        54100        56268       56933        57757        58013
decote_gain_fiscal                     -0.000000   -0.000000    6.172394    -0.000000    -0.000000   -0.000000    -0.000000    -0.000000   -0.000000    -0.000000   21.407501    -0.000000    -0.000000    -0.000000    -0.000000   18.437500    -0.000000    -0.000000    -0.000000   10.077515    -0.000000    -0.000000    -0.000000   -0.000000    -0.000000    -0.000000    -0.000000    -0.000000    -0.000000   -0.000000    -0.000000    -0.000000
decote_gain_fiscal_mensuel_times_12   172.035004  464.760040  416.992401   547.331299   793.025024  239.949997   103.684998   990.494934  262.387482   299.664978  576.510010   306.380005   187.995026   232.344971  1267.065186  862.657471    94.934998   224.864990   919.082397  765.180054   111.975037   527.119995   280.744995  969.153564   446.197540   397.440033   452.879974  1209.772583   651.937500  317.745117   443.804993   310.175018
irpp                                -3622.290039 -733.590088 -702.002808 -1114.052490 -2237.489990   58.500000 -3217.129883 -1401.330322 -899.690002 -1855.189941 -801.777466 -1764.589966 -1958.109985 -1652.910034 -1363.110229  339.312500 -4132.730957 -1112.930054 -1637.970215 -438.767456 -3227.070068  1929.894775 -3207.510010 -982.032593 -1287.850098 -2400.229980 -1719.830322 -1407.052490 -1294.025024   77.500000 -3298.330078 -2238.110107
irpp_mensuel                        -3607.883301 -728.357361 -685.938049 -1100.291870 -2231.991455   78.473022 -3208.571045 -1357.918701 -890.336853 -1849.975342 -775.151550 -1739.058594 -1942.443115 -1647.258911 -1353.106201  392.790649 -4124.737793 -1094.682007 -1596.960205 -406.640015 -3217.773926  1973.821533 -3198.975830 -967.489746 -1279.417725 -2383.302734 -1712.562500 -1338.503174 -1266.717163   91.466568 -3289.410889 -2212.168945
irpp_1                               -381.599152 -111.505257  -67.784019  -117.652725  -298.459167    0.111259   -46.245419   -44.652920  -82.159309  -197.975830  -77.515160  -236.515015  -196.993576   -43.123547   -36.970417   11.791809   -48.432919  -115.239830  -281.637848   -7.200836  -336.585297   147.332169  -312.974518  -32.495731  -167.065826   -39.046665  -178.809158   -21.759478   -16.692707   -1.945948  -353.762482  -275.554169
irpp_2                               -381.599152 -111.505257  -67.784019  -117.652725  -298.459167    0.111259  -287.484161   -44.652920  -82.159309  -197.975830  -77.515160  -236.515015  -196.993576   -43.123547   -36.970417   11.791809  -370.573120  -115.239830  -281.637848   -7.200836  -336.585297   250.250000  -312.974518  -32.495731  -167.065826   -39.046665  -178.809158   -21.759478   -16.692707   -1.945948  -353.762482   -56.657917
irpp_3                               -381.599152 -111.505257  -67.784019  -117.652725  -298.459167    0.111259  -287.484161   -44.652920  -82.159309  -197.975830  -77.515160  -236.515015  -196.993576  -156.101166   -36.970417   11.791809  -370.573120  -115.239830  -281.637848   -7.200836  -336.585297   250.250000  -312.974518  -32.495731  -167.065826  -251.795837  -178.809158   -21.759478   -16.692707   -1.945948  -353.762482   -56.657917
irpp_4                               -381.599152 -111.505257  -67.784019  -117.652725  -298.459167    0.111259  -287.484161   -44.652920  -82.159309  -197.975830  -77.515160  -236.515015  -196.993576  -156.101166   -36.970417   11.791809  -370.573120  -115.239830  -281.637848   -7.200836  -336.585297   147.332169  -312.974518  -32.495731  -167.065826   -39.046665  -178.809158   -21.759478  -135.182098   -1.945948  -353.762482   -56.657917
irpp_5                               -381.599152 -111.50

In [56]:
print "il y a {} cas ou l'impôt annuel est supérieur au mensuel".format(df_ir_sup_ir_m.shape[0])

il y a 32 cas ou l'impôt annuel est supérieur au mensuel


#### Différence due à la décote

In [48]:
df_ir_sup_ir_m.irpp + df_

decote_gain_fiscal  decote_gain_fiscal_mensuel_times_12          irpp  irpp_mensuel
4               26.372473                             0.000000      0.000000  -1203.458374
10               0.000000                           468.146576    153.500000    126.203308
11              -0.000000                             0.000000  -2335.952637  -8637.343750
12               0.000000                             0.000000      0.000000   -521.972473
16             116.709999                             0.000000      0.000000   -462.647491
54             279.206268                          3350.475342      0.000000    -28.381193
65             229.982498                          1510.709595   -176.052505   -488.322540
66              -0.000000                             0.000000 -25070.474609 -39161.695312
67              -0.000000                          1643.771729   -950.567383  -1734.394775
68               0.000000                             0.000000      0.000000   -619.029175
74             133.872498                             0.000000   -464.382507  -1367.037476
80              -0.000000                            -0.000000  -2422.667725  -2671.306885
91              -0.000000                            -0.000000   -450.965210  -1007.324890
98              -0.000000                             0.000000  -2660.685059 -18351.488281
108            240.501251                             0.000000      0.000000   -965.947571
115            281.172485                            -0.000000      0.000000  -1146.494995
120             -0.000000                             0.000000   1688.893066    681.849731
123            241.380005                          2323.019775   1216.640015   1073.255005
126             -0.000000                             0.000000   -118.244873  -1140.323486
127              0.000000                             0.000000      0.000000   -883.929199
132            132.260040                           911.579834   -255.219879   -424.105103
140             -0.000000                            -0.000000  -1231.369873  -1289.330078
142             41.222542                           741.494873   1150.000000    469.373688
153            249.712433                          2790.960693      0.000000    -51.397202
157            227.252563                             0.000000   -184.242310   -741.326172
158            280.649963                            -0.000000      0.000000   -916.054871
159             83.599976                             0.000000     69.500000   -210.421295
160             -0.000000                            -0.000000  -4381.064941  -4708.944824
162             93.342499                             8.756897   -585.972534   -791.644104
172              0.000000                          1663.232544      0.000000    -89.358513
188             -0.000000                            -0.000000  -1009.587402  -2061.827637
191              0.000000                             0.000000      0.000000   -467.503723
192             -0.000000                             0.000000  -2050.927734 -10437.636719
199             -0.000000                             0.000000 -16194.775391 -27589.394531
202            225.333771                          2440.004883   -189.998688   -255.998795
209             -0.000000                            -0.000000  -2726.475098  -3065.774414
210             -0.000000                            -0.000000  -1147.755249  -1532.055054
211             46.285004                            -0.000000   -688.145020  -1264.889771
213             -0.000000                             0.000000  -8115.674805 -12033.586914
218            243.842529                          2323.436279     48.000000    -20.640949
219             89.072510                             0.000000   -598.782471  -3112.264648
221            269.774933                           778.552185      0.000000   -454.861969
223            115.142487                          2831.387451      0.000000    -13.819759
228

In [17]:
df_ir_sup_ir_m.T

4           10           11          12          16           54           65            66           67          68           74           80          91            98          108          115          120          123          126         127         132         136          140          142          153         157         158         159          160         162          172          188         191           192           199          202          209          210          211           213          218          219         221          223          228          251         252          255          256          267          271          273           276          280          295          300        305         306         309         311          313         315          317          326          331          332           336           340         344          345          357          359          361          362         365          372          380          388          390          395          404          407         409          410          418          420          427          428          429         431           434          440          442          448          454          459          465          467          471          474      ...           59459        59460        59463        59465       59466       59472        59478       59480       59482        59483        59487       59488        59495        59504       59508        59512        59518        59528        59539       59540       59541         59544        59549       59550        59551        59552         59574       59579    59580        59583       59588       59591       59596       59598        59612        59615        59622        59631        59632        59636        59644       59645        59647       59648        59653        59654        59660       59667       59670       59672        59679        59681         59687        59688        59699        59701        59703        59704         59706       59709        59715         59718       59726        59727       59733        59739       59740       59742         59743         59746        59748        59752       59754        59768       59778        59779        59785        59796        59797        59800        59802       59804       59806        59807        59808       59814       59822        59825        59830        59831        59840        59846        59847        59851        59858        59860        59864        59865        59867      59871
decote_gain_fiscal                     26.372473    0.000000    -0.000000    0.000000  116.709999   279.206268   229.982498     -0.000000    -0.000000    0.000000   133.872498    -0.000000    -0.00000     -0.000000  240.501251   281.172485    -0.000000   241.380005    -0.000000    0.000000  132.260040    0.000000    -0.000000    41.222542   249.712433  227.252563  280.649963   83.599976    -0.000000   93.342499     0.000000    -0.000000    0.000000     -0.000000     -0.000000   225.333771    -0.000000    -0.000000    46.285004     -0.000000   243.842529    89.072510  269.774933   115.142487    -0.000000    -0.000000  167.472626   186.009995    39.862488    -0.000000   166.705002    44.823730     -0.000000    -0.000000    -0.000000    42.242493   166.5625  244.557480   39.862488   41.331238    -0.000000  275.067505    -0.000000    -0.000000    -0.000000   182.032425     -0.000000     -0.000000   44.397461   209.247467    -0.000000   119.307526   223.821243     0.000000    0.000000   199.167526   277.157471   223.052429    -0.000000    -0.000000    -0.000000   179.312424    0.000000   226.034973    -0.000000     0.662506    30.123932   273.981201    -0.000000   63.399994     -0.000000    -0.000000    -0.000000    -0.000000   253.589996    -0.000000   234.271240   260.369995    -0.000000   288.322510    ...      183.755005    30.262512    92.247528   162.502441  130.020004  177.264999    -0.000000    0.000000    0.000000  

In [ ]:
df.irpp

In [ ]:
survey_scenario.create_data_frame_by_entity_key_plural(['irpp','ip_net', ])['foyers_fiscaux']

In [ ]:
survey_scenario.compute_aggregate('irpp')/10**9

In [ ]:
survey_scenario.compute_aggregate('irpp')/10**9

In [ ]:
survey_scenario.compute_aggregate('irpp_mensuel')/10**9

In [ ]:
((impot_annuel == 0) & (impot_mensuel == 0)).sum()

In [ ]:
survey_scenario.compute_pivot_table(columns = ['nbF'], values = ["irpp"])

In [ ]:
df = survey_scenario.create_data_frame_by_entity_key_plural(['irpp','ip_net', ])['foyers_fiscaux']

In [ ]:
#### Modified Adrien


from numpy import (datetime64, logical_and as and_, logical_not as not_, logical_or as or_, logical_xor as xor_,
maximum as max_, minimum as min_, round)


lambda_compensation = simulation.calculate_add("lambda_compensation")
import pandas as pd
lambda_compensation = pd.Series(lambda_compensation)
print lambda_compensation.isnull().value_counts() #on devrait avoir autant de lambda compensation null que d'IR

higher_mensual = pd.Series((simulation.calculate_add("irpp_mensuel")> simulation.calculate_add('irpp'))).value_counts(dropna = False)
print "There is {} higher mensual then annual IR (not supposed to happened)".format(higher_mensual)

impot_annuel = simulation.calculate("irpp")
impot_mensuel = simulation.calculate_add("irpp_mensuel")
print "il y a {}/{} cas ou l'impot annuel et mensuel est null soit {}%".format(((impot_annuel == 0) & (impot_mensuel == 0)).sum(),impot_mensuel.shape[0],((impot_annuel == 0) & (impot_mensuel == 0)).sum()/impot_mensuel.shape[0] )
print "il y a {}/{} cas ou l'impot mensuel est null, mais pas l'annuel soit {}% (pas possible ?)".format(((impot_annuel != 0) & (impot_mensuel == 0)).sum(), impot_mensuel.shape[0],((impot_annuel != 0) & (impot_mensuel == 0)).sum()/impot_mensuel.shape[0])
print "il y a {}/{} cas ou l'impot mensuel est inférieur à l'annuel soit {}% (pas possible ?)".format((impot_annuel > impot_mensuel).sum(), impot_mensuel.shape[0], (impot_annuel > impot_mensuel).sum()/ impot_mensuel.shape[0])


irpp = pd.Series(simulation.calculate("irpp"), name = "irpp")
irpp_mensuel = pd.Series(simulation.calculate("irpp_mensuel"),name = 'irpp_mensuel')
decote = pd.Series(simulation.calculate("decote"), name = "decote")
ir_plaf_qf = pd.Series(simulation.calculate("ir_plaf_qf"))
decote = min_(decote, ir_plaf_qf)


ir_plaf_qf_m = pd.Series(simulation.calculate_add("ir_plaf_qf_mensuel_times_12"))
decote_m = pd.Series(simulation.calculate_add("decote_mensuel_times_12"))
decote_m = min_(decote_m, ir_plaf_qf_m)/12

print u"le gain fiscal de la décote est {}% plus élevé en mensuel".format(((decote_m.sum()/decote.sum()-1))*100)




ppe = pd.Series(simulation.calculate("ppe"), name="ppe")

#Qui sont ceux qui payent de l'impôt en trop en mensuel ?


#7657 ;
annomalie = (impot_mensuel<impot_annuel +1) | (impot_mensuel<impot_annuel - 1) #gérer les tout petits problèmes d'arrondis

print u"{}% payent plus en annuel à cause de la décote (ou tout du moins en partie)".format((decote_m[annomalie]==0).value_counts(normalize = True)[0])
print (decote_m[annomalie]==0).value_counts(normalize = True)
#D'ou viennent les autres annomalies, pourquoi plus d'ir en annuel



df_ir_mois = pd.DataFrame()
for mois in range(1,13):
    tempo = pd.Series(simulation.calculate('irpp_mensuel', '2009-{}'.format(mois)))
    tempo.name = 'irpp_mois_{}'.format(mois)
    df_ir_mois = df_ir_mois.append(tempo)

df_ir_mois = df_ir_mois.append(irpp)
df_ir_mois = df_ir_mois.append(irpp_mensuel)

decote.name = "decote"
df_ir_mois = df_ir_mois.append(decote)

decote_m.name = "decote_m"
df_ir_mois = df_ir_mois.append(decote_m)


df_ir_mois.to_hdf('/Users/adrienpacifico/Desktop/to_trash.h5', 'df_ir_mois')


mr = 17352
